In [13]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
import numpy as np 
import skimage
from skimage import transform
from skimage import io 
from matplotlib import pyplot as plt
%matplotlib inline

In [14]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [16]:
#import keras

In [17]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [18]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [19]:
x_train[0:32].shape

(32, 32, 32, 3)

In [20]:
%time model.train_on_batch(x_train[0:32], y_train[0:32])

CPU times: user 55.9 ms, sys: 36.9 ms, total: 92.8 ms
Wall time: 153 ms


[array(2.3009143, dtype=float32), array(0.0625, dtype=float32)]

In [21]:
#help(model.train_on_batch)

In [22]:
%%time 
epochs = 1
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
49984/50000 [============================>.] - ETA: 0s - loss: 1.8020 - acc: 0.3403

INFO:plaidml:Analyzing Ops: 135 of 320 operations complete


50000/50000 [==============================] - 143s 3ms/step - loss: 1.8019 - acc: 0.3404 - val_loss: 1.5727 - val_acc: 0.4394
CPU times: user 27.3 s, sys: 22.8 s, total: 50.1 s
Wall time: 2min 23s


In [23]:
%%time 
epochs = 1
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 140s 3ms/step - loss: 1.4779 - acc: 0.4640 - val_loss: 1.3251 - val_acc: 0.5254
CPU times: user 24.6 s, sys: 22.9 s, total: 47.5 s
Wall time: 2min 20s
